# Домашнее задание 3
## Yes/No Questions

deadline: 4 декабря 2022, 23:59 МСК

В этом домашнем задании вы будете работать с корпусом DaNetQA. Корпус состоит из вопросов, предполагающих бинарный ответ (да / нет), абзацев из Википедии, содержащих ответ на вопрос, и непосредственно ответа (true / false).

Корпус описан в статье:

Glushkova, T., Machnev, A., Fenogenova, A., Shavrina, T., Artemova, E., & Ignatov, D. I. (2020, October). Danetqa: a yes/no question answering dataset for the russian language. In International Conference on Analysis of Images, Social Networks and Texts (pp. 57-68). Springer, Cham.

https://arxiv.org/abs/2010.02605


Корпус (train-val-test split) доступен по ссылке: https://russiansuperglue.com/ru/tasks/task_info/DaNetQA

Используйте для обучения train часть корпуса, для валидации val и тестирования – test часть. 

Каждый бонус пункт оценивается в 1 балл. 

### Пример вопроса: 
question: Был ли человек на луне?

label (answer): true

text (passage): В период с 1969 по 1972 год по программе «Аполлон» было выполнено 6 полётов с посадкой на Луне. Всего на Луне высаживались 12 астронавтов США. Список космонавтов Список космонавтов — участников орбитальных космических полётов Список астронавтов США — участников орбитальных космических полётов Список космонавтов СССР и России — участников космических полётов Список женщин-космонавтов Список космонавтов, посещавших МКС Энциклопедия астронавтики.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо в ipython-тетрадке. При этом ipynb-файл прилагается обязательно. 
3. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросовестное цитирование приводит к обнулению оценки. 

## Часть 1. [1 балл] Эксплоративный анализ
1. Посчитайте долю yes и no классов в корпусе.
2. Оцените среднюю длину вопроса.
3. Оцените среднюю длину параграфа.
4. Предположите, по каким эвристикам были собраны вопросы. Продемонстрируйте, как эти эвристики повлияли на структуру корпуса.

In [1]:
import json
import pandas as pd
import numpy as np

In [7]:
train_path = "data/train.jsonl"
val_path = "data/val.jsonl"
test_path = "data/test.jsonl"

def read_data(path_data):
    file = open(path_data, 'r')
    lines = file.readlines()
    data = []
    for line in lines:
        data.append(json.loads(line))
    return data

train_data = read_data(train_path)
val_data = read_data(val_path)
test_data = read_data(test_path)

In [8]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

1749
821
805


In [13]:
df_train = pd.DataFrame.from_records(train_data)
df_val = pd.DataFrame.from_records(val_data)
df_test = pd.DataFrame.from_records(test_data)

In [19]:
df_train.head()

,question,passage,label,idx
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True,0
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True,1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True,2
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True,3
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False,4


In [33]:
df_train["label"] = df_train["label"].apply(lambda x: 1 if x else 0)
df_train = df_train.drop(["idx"], axis=1)
df_val["label"] = df_val["label"].apply(lambda x: 1 if x else 0)
df_val = df_val.drop(["idx"], axis=1)
df_test = df_test.drop(["idx"], axis=1)

In [26]:
df_train.head()

,question,passage,label
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,1
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,1
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",1
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,0


In [42]:
print(f'mean len for passages: {df_train["passage"].apply(len).mean()}')
print(f'mean len for questions: {df_train["question"].apply(len).mean()}')

mean len for passages: 723.1149228130361
mean len for questions: 41.63064608347627


In [47]:
df_train[["question", "passage"]].head(30)

,question,passage
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,..."
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010..."
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...
5,Может ли автомобиль ездить на газу?,Автомобиль на природном газе — один из видов а...
6,Может ли автомобиль ездить на газу?,Для работы на газообразных топливах транспортн...
7,Был ли автомобиль принцессы дианы в дтп?,Несмотря на продолжительные реанимационные поп...
8,Есть ли в индийском океане акулы?,"Обыкновенная акула-молот, или молот-рыба — од..."
9,Есть ли в индийском океане акулы?,Ри́фовая аку́ла — единственный вид рода рифов...


Видим, что вопросы все достаточно похожи по своей структуре. Скорее всего они построены на основе каких-либо шаблонов. Авторы в своей статье пишут, что для создания вопросов они использовали Яндекс Толоку, где указали около 50 шаблонов для создания вопросов к тексту. Ну и плюс они все являются да/нет вопросами из-за этого понятно почему у них более менее схожая структура

In [52]:
train_data[:2]

[{'question': 'Вднх - это выставочный центр?',
  'passage': '«Вы́ставочный центр» — станция Московского монорельса. Расположена между станциями «Улица Академика Королёва» и «Улица Сергея Эйзенштейна». Находится на территории Останкинского района Северо-Восточного административного округа города Москвы. Переход на станцию  ВДНХ Калужско-Рижской линии. Названа в честь Всероссийского выставочного центра — названия ВДНХ с 1992 по 2014 год. 20 ноября 2004 года линия монорельса начала работать в «экскурсионном режиме» и перевезла первых пассажиров .',
  'label': True,
  'idx': 0},
 {'question': 'Вднх - это выставочный центр?',
  'passage': 'Вы́ставка достиже́ний наро́дного хозя́йства  , в 1959—1991 годах — Вы́ставка достиже́ний наро́дного хозя́йства СССР , в 1992—2014 годах — Всеросси́йский вы́ставочный центр ) — выставочный комплекс в Останкинском районе Северо-Восточного административного округа города Москвы, второй по величине выставочный комплекс в городе. Входит в 50 крупнейших выставо

Кажется, что passage взяты с Википедии. Авторы в статье описали каким образом они собирали passage: с помощью Google API они отправляли полученный с толоки вопрос и находили несколько страниц с Wikipedia. Далее с помощью предобученныего Bert они могли извлекать нужный кусок текста для ответа на вопрос.

## Часть 2. [1 балл] Baseline
1. Оцените accuracy совсем простого базового решения: присвоить каждой паре вопрос-ответ в test части самый частый класс из train части.
2. Оцените accuracy чуть более сложного базового решения: fasttext на текстах, состоящих из склеенных вопросов и абзацев (' '.join([question, passage])).

Сравните полученные результаты и опишите, почему они получились именно такими.

## Часть 3. [3 балла] Используем эмбеддинги предложений
1. Постройте BERT эмбеддинги вопроса и абзаца на основе двух моделей: векторизации русского языка и multilingual модели векторизации. Обучите логистическую регрессию на конкатенированных эмбеддингах вопроса и абзаца и оцените различия в accuracy моделей, обученных на векторах "разного происхождения". 

2. Векторизуйте данные корпуса BoolQ (https://github.com/google-research-datasets/boolean-questions) с помощью multilingual модели из п. 1 и аналогично обучите логистическую регрессию на конкатенированных эмбеддингах вопроса и абзаца. Оцените её качество (обучаем на train, проверяем на test).

3. Примените модель из п. 2 к данным DaNetQA и наоборот - модель на основе multilingual векторизации, обученную на DaNetQA, к данным BoolQ. Сравните полученные результаты по точности предсказаний.

4. Объедините train в единую обучающую выборку (взять конкатенированные multilingual эмбеддинги) и обучите модель по аналогии с п. 1. Рассчитайте точность на обеих тестовых подвыборках отдельно и сравните результаты с теми, которые были получены в рамках предыдущих пунктов.


[бонус] Используйте другие модели эмбеддингов, доступные, например, в библиотеке Transformers. Какая модель эмбеддингов даст лучшие результаты?

[бонус] Предложите метод аугментации данных и продемонстрируйте его эффективность. 

## Часть 3. [4 балла] Модель на основе архитектуры Transformer

Выберите две модели на основе архитектуры transformer (BERT, RoBERTa, XLM etc.) - русскоязычную и мультиязычную и дообучите их на корпусах DaNetQA и BoolQ. 

Оцените качество этих моделей для решения задачи. Сравните их работоспособность между собой на основе метрик качества


## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?